### exploring pose estimation

In [ ]:
import os
import cv2
from ultralytics import YOLO
import numpy as np

# --- CONFIG ---
video_path = "dataset/tacticam.mp4"  # or "dataset/broadcast.mp4"
output_dir = "output_pose_overlay"
os.makedirs(output_dir, exist_ok=True)
max_frames = 5

yolo_model = YOLO("models/best.pt")
pose_model = YOLO("yolov8n-pose.pt")  # Download from Ultralytics if needed

cap = cv2.VideoCapture(video_path)
frame_count = 0
font = cv2.FONT_HERSHEY_SIMPLEX

while frame_count < max_frames:
    ret, frame = cap.read()
    if not ret:
        print("End of video.")
        break

    # --- Detect players ---
    det_result = yolo_model.predict(source=frame, conf=0.75, save=False, verbose=False)[0]
    annotated = frame.copy()

    # Create frame directory
    frame_dir = os.path.join(output_dir, f"frame_{frame_count}")
    os.makedirs(frame_dir, exist_ok=True)

    if det_result.boxes is not None:
        for i, box in enumerate(det_result.boxes):
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            cls_id = int(box.cls[0].item())
            class_name = yolo_model.names[cls_id]
            if class_name.lower() != "player":
                continue

            # --- Crop player and run pose estimation on crop ---
            crop = frame[y1:y2, x1:x2]
            pose_result = pose_model.predict(source=crop, conf=0.3, save=False, verbose=False)[0]

            # Overlay pose keypoints on the crop
            crop_pose = crop.copy()
            if pose_result.keypoints is not None and len(pose_result.keypoints) > 0:
                for kp in pose_result.keypoints.xy:
                    for (px, py) in kp:
                        cv2.circle(crop_pose, (int(px), int(py)), 3, (0, 255, 255), -1)

            # Save player crop with pose overlay
            crop_pose_path = os.path.join(frame_dir, f"player_{i}_pose.jpg")
            cv2.imwrite(crop_pose_path, crop_pose)

            # Paste the crop with pose overlay back to the annotated frame
            annotated[y1:y2, x1:x2]

End of video.


## using OpenPifPaf